# Redes Neuronales Artificiales (ANN) vs. Redes Neuronales Informadas por la Física (PINNs)

**Autores:** Tabita Catalán, Tomás Banduc, David Ortiz y Francisco Sahli — 2025

Accede al trabajo fundacional de las PINNs [aquí](https://www.sciencedirect.com/science/article/pii/S0021999118307125).

### Introducción

En la lección anterior se construyó una PINN básica siguiendo un esquema estructurado de seis pasos, aplicado a un sistema lineal clásico (masa–resorte–amortiguador) con solución analítica conocida. Ese ejercicio permitió introducir los conceptos fundamentales de las PINNs, así como el rol de la física en la función de pérdida.

En esta segunda lección se amplía ese marco comparando directamente Redes Neuronales Artificiales (ANNs) y Redes Neuronales Informadas por la Física (PINNs). Mientras que las ANNs dependen exclusivamente de los datos para aprender una solución, las PINNs incorporan de forma explícita las ecuaciones gobernantes del sistema, lo que mejora la estabilidad, reduce la dependencia de grandes conjuntos de datos y aporta interpretabilidad física.

La comparación se realiza sobre el modelo del péndulo oscilante, un sistema no lineal que representa un aumento natural de complejidad respecto al caso estudiado previamente.

### Resumen de la Actividad

En esta actividad se implementan dos aproximaciones para resolver el modelo matemático no lineal de un péndulo oscilante:

- una Red Neuronal Artificial (ANN) entrenada únicamente a partir de datos,
- y una Red Neuronal Informada por la Física (PINN), siguiendo los mismos seis pasos introducidos en la lección anterior.

Este enfoque permite aislar y analizar el efecto de incorporar la física del sistema en la función de pérdida, comparando desempeño, estabilidad y capacidad de generalización entre ambos modelos.

### Objetivos de la Actividad

Al finalizar esta actividad, podrás:

- Comprender las diferencias conceptuales y prácticas entre ANNs y PINNs.  
- Analizar por qué las PINNs requieren menos datos y presentan mayor coherencia física.  
- Implementar y entrenar ANNs y PINNs en PyTorch para un mismo problema.  
- Evaluar el impacto de la física en la calidad de la solución aprendida.  


## Modelo matemático para describir un péndulo oscilante

Queremos resolver el problema matemático relacionado con el **péndulo oscilante** [(wiki)](https://en.wikipedia.org/wiki/Pendulum_(mechanics)):

| ![GIF](../figures/Oscillating_pendulum.gif?raw=1) | <img src="../figures/Pendulum_gravity.svg?raw=1" alt="Diagrama del proyecto" width="300"/> |
|-------------------------------------------|-------------------------------------------|
| Vectores de velocidad y aceleración del péndulo  | Diagrama de fuerzas |


**Supuestos:**

- La varilla es rígida y sin masa [(Tarea - el caso de una cuerda elástica)](https://en.wikipedia.org/wiki/Elastic_pendulum#:~:text=In%20physics%20and%20mathematics%2C%20in,%2Ddimensional%20spring%2Dmass%20system.).
- El peso es una masa puntual.  
- Dos dimensiones [(Tarea - una dimensión adicional de movimiento)](https://www.instagram.com/reel/CffUr64PjCx/?igsh=MWlmM2FscG9oYnp6bw%3D%3D).
- No hay resistencia del aire [(Tarea - inmersión en un fluido)](https://www.youtube.com/watch?v=erveOJD_qv4&ab_channel=Lettherebemath).
- El campo gravitacional es uniforme y el soporte no se mueve.

Nos interesa encontrar el ángulo vertical $\theta(t) \in [0, 2\pi)$ tal que:

$$
\frac{d^2\theta}{dt^2}+\frac{g}{l}\sin\theta=0,\quad\theta(0)=\theta_0,\quad\theta'(0)=0,\quad t\in\mathbb{R},
$$

donde $g\approx 9.81[m/s^2]$, $l$ es el largo de la varilla y $t$ la variable temporal.  

**Repaso de conceptos de ecuaciones diferenciales:**

- ¿Por qué esta es una ecuación diferencial no lineal? ¿Qué supuestos deberían hacerse para linealizar el modelo?
- ¿Es una ecuación diferencial ordinaria (EDO) o una ecuación diferencial parcial (EDP)?  
- ¿Cuál es el orden? ¿cuál es el grado?  

Un método útil es convertir el modelo en un sistema acoplado de EDOs:  

$$
\begin{align*}
\frac{d\theta}{dt} &= \omega, \quad \text{(velocidad angular)}\\
\frac{d\omega}{dt} & = -\frac{g}{l}\sin\theta, \quad \text{(aceleración angular)}
\end{align*}
$$

### Flujo de Trabajo  
1. Calcular la solución numérica del modelo no lineal del péndulo oscilante y preparación de los datos de entrenamiento añadiendo ruido, remuestreando y limitando el tiempo para simular un escenario real.  
2. Definir el modelo ANN utilizando la arquitectura de PyTorch y entrenar con los datos preparados. Graficar la solución.  
3. Definir el modelo PINN utilizando la arquitectura de PyTorch y entrenar con los datos preparados. Graficar la solución.  
4. Comparar las soluciones obtenidas con ambas arquitecturas. 

## Configuración Inicial  

Comenzamos importando algunos paquetes útiles y definiendo algunas funciones.

In [ ]:
%matplotlib inline

In [ ]:
# Import NumPy for numerical operations
import numpy as np
# Import PyTorch for building and training neural networks
import torch
import torch.nn as nn
import torch.optim as optim
# Import Matplotlib for plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
import time

# Ignore Warning Messages
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Setup (device + plots)
def get_device() -> str:
    return "cuda" if torch.cuda.is_available() else \
           "mps"  if torch.backends.mps.is_available() else "cpu"

def set_mpl_style(gray: str = "#5c5c5c") -> None:
    mpl.rcParams.update({
        "image.cmap": "viridis",
        "text.color": gray, "xtick.color": gray, "ytick.color": gray,
        "axes.labelcolor": gray, "axes.edgecolor": gray,
        "axes.spines.right": False, "axes.spines.top": False,
        "axes.formatter.use_mathtext": True, "axes.unicode_minus": False,
        "font.size": 15, "interactive": False, "font.family": "sans-serif",
        "legend.loc": "best", "text.usetex": False, "mathtext.fontset": "stix",
    })

device = get_device()
print(f"Using {device} device")
set_mpl_style()

# Metrics
def calculate_snr(signal, noise) -> float:
    s, n = np.asarray(signal), np.asarray(noise)
    return 10.0 * np.log10(np.mean(s**2) / np.mean(n**2))

def relative_l2_error(u_num: torch.Tensor, u_ref: torch.Tensor) -> torch.Tensor:
    return torch.norm(u_num - u_ref) / torch.norm(u_ref)

# Autodiff helper
def grad(outputs: torch.Tensor, inputs: torch.Tensor) -> torch.Tensor:
    """d(outputs)/d(inputs) with create_graph=True."""
    return torch.autograd.grad(
        outputs, inputs,
        grad_outputs=torch.ones_like(outputs),
        create_graph=True
    )[0]

# Plotting
def plot_comparison(t: torch.Tensor, theta_true, theta_pred: torch.Tensor, loss) -> None:
    t_np = t.detach().cpu().numpy().ravel()
    pred_np = theta_pred.detach().cpu().numpy().ravel()
    true_np = np.asarray(theta_true).ravel()
    diff = np.abs(true_np - pred_np)

    fig, axs = plt.subplots(1, 2, figsize=(12, 5))
    axs[0].plot(t_np, true_np, label=r'$\theta(t)$ (numerical)')
    axs[0].plot(t_np, pred_np, label=r'$\theta_{\mathrm{pred}}(t)$')
    axs[0].set(title='Numerical vs Predicted', xlabel=r'Time $(s)$', 
               ylabel='Amplitude', ylim=(-1, 1.3))
    axs[0].legend(frameon=False)

    axs[1].plot(t_np, diff)
    axs[1].set(title='Absolute Difference', xlabel=r'Time $(s)$', 
               ylabel=r'$|\theta - \theta_{\mathrm{pred}}|$')
    fig.tight_layout()
    plt.show()

    fig, ax = plt.subplots(1, 1, figsize=(6, 3))
    ax.plot(loss)
    ax.set(title='Training Progress', xlabel='Iteration', 
           ylabel='Loss', xscale='log', yscale='log')
    ax.grid(True)
    fig.tight_layout()
    plt.show()

## 1. Solución numérica del péndulo oscilante y preparación de los datos de entrenamiento
El objetivo de esta etapa es simular la obtención de datos de un péndulo, considerando los supuestos previamente mencionados.

### 1.1 Solución numérica
Para la solución numérica utilizamos el método [Runge-Kutta de cuarto orden](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods) de `scipy`. Comenzamos definiendo los parámetros para este ejemplo, el modelo del péndulo y el dominio

In [ ]:
g = 9.81  # gravity acceleration (m/s^2)
L = 1.0   # Pendulum's rod length (m)
theta0 = np.pi / 4  # Initial condition (Position in rads)
omega0 = 0.0        # Initial angular speed (rad/s)
sample_freq = 100   # sample rate 100Hz

# Simulation time
t_span = (0, 10)  # from 0 to 10 seconds
t_eval = np.linspace(t_span[0], t_span[1], sample_freq*t_span[1])  # Points to be evaluated

# We define the system of coupled ODEs
def pendulum(t, y):
    theta, omega = y
    dtheta_dt = omega
    domega_dt = -(g / L) * np.sin(theta)
    return [dtheta_dt, domega_dt]

# Initial conditions
y0 = [theta0, omega0]

Ahora resolvemos el problema numéricamente utilizando `scipy`.

In [ ]:
from scipy.integrate import solve_ivp

# Solve the initial value problem using Runge-Kutta 4th order
num_sol = solve_ivp(pendulum, t_span, y0, t_eval=t_eval, method='RK45')

# We extract the solutions
theta_num = num_sol.y[0]
omega_num = num_sol.y[1]

# We graph the results
plt.figure(figsize=(12, 6))
plt.plot(t_eval, theta_num, label=r'Angular Displacement $\theta(t)[rad]$')
plt.plot(t_eval, omega_num, label=r'Angular Velocity $\omega(t)[rad/s]$')
plt.xlabel(r'Time $[s]$')
plt.ylim(-2.5,3.3)
plt.legend(loc='best', frameon=False)
plt.title('Nonlinear Pendulum Solution')
plt.grid(False)
plt.tight_layout()
plt.show()

### 1.2. Preparación de los datos de entrenamiento  <a id='data_prep'></a>

A continuación, consideramos la solución numérica como los **datos de entrenamiento** que provienen de las mediciones de un sensor. Añadimos ruido gaussiano, remuestreamos y recortamos los datos a $2.5$ s para evaluar el rendimiento de la ANN, simulando un escenario real. Además, calculamos la relación señal-ruido $SNR = 10\log_{10} \left(\frac{P_{signal}}{P_{noise}}\right)$, donde $P_{signal}$ y $P_{noise}$ son la potencia de la señal y del ruido, respectivamente, para determinar el nivel de distorsión en la señal. Finalmente, llamamos a los datos de entrenamiento con ruido $\theta_{data}(t)$.  


In [ ]:
# Add gaussian noise
std_deviation = 0.05
noise = np.random.normal(0,std_deviation,theta_num.shape[0])
theta_noisy = theta_num + noise
print(f'SNR: {calculate_snr(theta_noisy, noise):.4f} dB')

# Resample and cut to 2.5s
resample = 5          # resample
cut_time = int(2.5*sample_freq)  # 2.5s times 100Hz

theta_data = theta_noisy[:cut_time:resample]
t_data = t_eval[:cut_time:resample]

# We graph the observed data
plt.figure(figsize=(12, 6))
plt.plot(t_eval, theta_num, label=r'Angular Displacement (model) $\theta(t)$ ')
plt.plot(t_data, theta_data, label=r'Training data (measures) $\theta_{data}(t)$ ')
plt.xlabel(r'Time $[s]$')
plt.ylabel(r'Angular displacement $[rad]$')
plt.ylim(-1,1.3)
plt.legend(loc='best', frameon=False)
plt.title('Training data')
plt.grid(False)
plt.show()

## 2. Entrenando la Red Neuronal Artificial

Entrenaremos la red neuronal artificial para aproximar directamente la solución de la ecuación diferencial, es decir,

$$
\theta_{NN}(t; \Theta) \approx \theta(t)
$$

donde $\Theta$ son los parámetros entrenables de la ANN. Utilizaremos `PyTorch` para definir la red y la entrenaremos con el optimizador ADAM. Además, convertiremos el dominio temporal y las observaciones a `torch.tensors`. 

### Función de pérdida 

Para entrenar la ANN necesitamos datos y una función de pérdida. Nuestros datos serán observaciones ruidosas de la solución $\theta_{data}(t)$, obtenidas en puntos de colocación $\{t_i\}_N$ elegidos del dominio. Utilizamos como función de pérdida el error cuadrático medio ($MSE$) entre estas observaciones y la evaluación de la ANN en los mismos puntos de colocación, es decir,

$$
\mathcal{L}(\Theta) := \lambda_1 MSE(\theta_{NN}(t; \Theta), \theta_{data}(t)) = \frac{\lambda_1}{N}\sum_i (\theta_{NN}(t_i; \Theta) - \theta_{data}(t_i))^2
$$

donde $\lambda_1 \in \mathbb{R}^+$ es un peso positivo y $N$ es el número de muestras. El entrenamiento se realiza minimizando la función de pérdida $\mathcal{L}(\Theta)$, es decir,

$$
\min_{\Theta \in \mathbb{R}} \mathcal{L}(\Theta) \rightarrow 0
$$

In [ ]:
#===============================================================================
# ETAPA 1: DEFINICIÓN DE LOS PARÁMETROS (MODELO FÍSICO)
#===============================================================================


#===============================================================================
# ETAPA 2: DEFINICIÓN DEL DOMINIO 
#===============================================================================


#===============================================================================
# ETAPA 3: CREACIÓN DE LA RED NEURONAL SURROGANTE 
#===============================================================================


#==========================================================================
# ETAPA 5: DEFINICIÓN DE LA FUNCIÓN DE COSTO BASADA ÚNICAMENTE EN LOS DATOS
#==========================================================================


#==========================================================================
# ETAPA 6: DEFINICIÓN DEl OPTIMIZADOR
#==========================================================================


#==========================================================================
# CICLO DE ENTRENAMIENTO
#==========================================================================


In [ ]:
torch.manual_seed(123)

# training parameters
hidden_layers = [1, 50, 50, 50, 1]
learning_rate = 0.001
training_iter = 50000

In [ ]:
# Define a loss function (Mean Squared Error) for training the network
MSE_func = nn.MSELoss()

# Convert the NumPy arrays to PyTorch tensors and add an extra dimension
# test time Numpy array to Pytorch tensor
t_test = torch.tensor(t_eval, device=device, requires_grad=True).view(-1,1).float()
# train time Numpy array to Pytorch tensor
t_data = torch.tensor(t_data, device=device, requires_grad=True).view(-1,1).float()
# Numerical theta to test Numpy array to pytorch tensor
theta_test = torch.tensor(theta_num, device=device, requires_grad=True).view(-1,1).float()
# Numerical theta to train Numpy array to pytorch tensor
theta_data = torch.tensor(theta_data, device=device, requires_grad=True).view(-1,1).float()

# Define a neural network class with user defined layers and neurons
class NeuralNetwork(nn.Module):

    def __init__(self, hlayers):
        super(NeuralNetwork, self).__init__()

        layers = []
        for i in range(len(hlayers[:-2])):
            layers.append(nn.Linear(hlayers[i], hlayers[i+1]))
            layers.append(nn.Tanh())
        layers.append(nn.Linear(hlayers[-2], hlayers[-1]))

        self.layers = nn.Sequential(*layers)
        self.init_params()

    def init_params(self):
        """Xavier Glorot parameter initialization of the Neural Network
        """
        def init_normal(m):
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight) # Xavier
        self.apply(init_normal)

    def forward(self, x):
        return self.layers(x)


In [ ]:
# Create an instance of the neural network
theta_ann = NeuralNetwork(hidden_layers).to(device)
nparams = sum(p.numel() for p in theta_ann.parameters() if p.requires_grad)
print(f'Number of trainable parameters: {nparams}')

# Define an optimizer (Adam) for training the network
optimizer = optim.Adam(theta_ann.parameters(), lr=learning_rate,
                       betas= (0.99,0.999), eps = 1e-8)

In [ ]:
def NeuralNetworkLoss(forward_pass, t, theta_data, lambda1 = 1):

    theta_nn = forward_pass(t)
    data_loss = lambda1 * MSE_func(theta_nn, theta_data)

    return  data_loss

# Initialize a list to store the loss values
loss_values_ann = []

# Start the timer
start_time = time.time()

# Training the neural network
for i in range(training_iter):

    optimizer.zero_grad()   # clear gradients for next train

    # input x and predict based on x
    loss = NeuralNetworkLoss(theta_ann,
                             t_data,
                             theta_data)    # must be (1. nn output, 2. target)

    # Append the current loss value to the list
    loss_values_ann.append(loss.item())

    if i % 1000 == 0:  # print every 100 iterations
        print(f"Iteration {i}: Loss {loss.item()}")

    loss.backward() # compute gradients (backpropagation)
    optimizer.step() # update the ANN weigths

# Stop the timer and calculate the elapsed time
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time} seconds")


graficamos los resultados

In [ ]:
theta_pred_ann = theta_ann(t_test).to(device)

print(f'Relative error: {relative_l2_error(theta_pred_ann, theta_test)}')

plot_comparison(t_test, theta_num, theta_pred_ann, loss_values_ann)

## 4. Entrenando la Red Neuronal Informada por Física (PINN)

Ahora queremos entrenar una PINN de para aproximar la solución de la EDO,

$$
\theta_{PINN}(t; \Theta) \approx \theta(t).
$$

Esta PINN tendrá la misma arquitectura que la ANN que usamos anteriormente, pero esta vez la entrenaremos incorporando la física del problema además de las observaciones. Usaremos las mismas observaciones ruidosas (**datos de entrenamiento**) que antes, pero también incluiremos las ecuaciones que describen el comportamiento de las derivadas de la solución.

In [ ]:
# Create an instance of the neural network
theta_pinn = NeuralNetwork(hidden_layers).to(device)
nparams = sum(p.numel() for p in theta_pinn.parameters() if p.requires_grad)
print(f'Number of trainable parameters: {nparams}')

# Define an optimizer (Adam) for training the network
optimizer = optim.Adam(theta_pinn.parameters(), lr=learning_rate,
                       betas= (0.99,0.999), eps = 1e-8)

### Función de pérdida informada por la física del problema

Para entrenar la PINN, recordemos el modelo del péndulo y definamos las funciones $f_{ode}(t;g,l)$, $g_{ic}(0)$ y $h_{bc}(0)$ para la ODE, la condición inicial y la condición de frontera. Además, reemplazamos la solución analítica $\theta(t)$ con la salida de la PINN $\theta_{pinn}(t; \Theta)$:

$$
\begin{align*}
f_{ode}(t;\theta_{pinn}):=&\frac{d^2\theta_{PINN}(t; \Theta)}{dt^2}+\frac{g}{l}\sin(\theta_{pinn}(t; \Theta)) = 0\\
g_{ic}(0;\theta_{pinn}):=&\theta_{pinn}(0; \Theta) = \theta_0\\
h_{bc}(0;\theta_{pinn}):=&\theta_{pinn}'(0; \Theta) = 0
\end{align*}
$$

Nuevamente utilizamos el $MSE$ y definimos la función de pérdida informada por física:

$$
\begin{align*}
\mathcal{L}(\theta):= &\frac{\lambda_1}{N}\sum_i\left(f_{ode}(t_i;\theta_{pinn})-0\right)^2 \quad \text{Pérdida de la ODE}\\
                   & + \lambda_2 (g_{ic}(0;\theta_{pinn})-\theta_0)^2 \quad \text{Pérdida de la condición inicial IC}\\
                   & + \lambda_3 (h_{bc}(0;\theta_{pinn})-0)^2 \quad \text{Pérdida de la condición de frontera BC}\\
                   & + \frac{\lambda_4}{N}\sum_i (\theta_{pinn}(t_i; \Theta) - \theta_{data}(t_i))^2 \quad \text{Pérdida de los datos (DATA)}
\end{align*}
$$

donde $\lambda_{1,2,3,4}\in\mathbb{R}^+$ son pesos positivos y $N$ es el número de muestras. 

<div class="alert alert-info"
    style="background-color:#5c5c5c;color:#000000;border-color:#000000">
  <strong>REMARK!</strong> Cuando no incluimos la función de pérdida relacionada con los datos, estamos empleando un esquema independiente de datos (*data-free*); cuando incluimos los datos, estamos empleando un esquema impulsado por datos (*data-driven*).
</div>

El entrenamiento se realiza minimizando la función de pérdida $\mathcal{L}(\Theta)$, es decir,

$$
\min_{\Theta\in\mathbb{R}} \mathcal{L}(\Theta)\rightarrow 0
$$

<div class="alert alert-info"
    style="background-color:#5c5c5c;color:#000000;border-color:#000000">
  <strong>REMARK!</strong> La diferenciación automática (`torch.autograd`) es una herramienta poderosa para calcular los gradientes de la PINN con respecto a la entrada, lo que será útil para evaluar la función de pérdida. 
</div>



In [ ]:
# Define t = 0 for boundary an initial conditions
t0 = torch.tensor(0., device=device, requires_grad=True).view(-1,1)

# HINT: use grad funtion (a wraper for torch.autograd) to calculate the
# derivatives of the ANN
def PINNLoss(forward_pass, t_phys, t_data, theta_data, 
             lambda1 = 1, lambda2 = 1, lambda3 = 1, lambda4 = 1):

    # ANN output, first and second derivatives
    theta_pinn1 = forward_pass(t_phys)
    theta_pinn_dt = grad(theta_pinn1, t_phys)
    theta_pinn_ddt = grad(theta_pinn_dt, t_phys)
    
    f_ode = theta_pinn_ddt + (g/L) * torch.sin(theta_pinn1)
    ODE_loss = lambda1 * MSE_func(f_ode, torch.zeros_like(f_ode)) 
    
    g_ic = forward_pass(t0)
    IC_loss = lambda2 * MSE_func(g_ic, torch.ones_like(g_ic)*theta0)
    
    h_bc = grad(forward_pass(t0),t0)
    BC_loss = lambda3 * MSE_func(h_bc, torch.zeros_like(h_bc))
    
    theta_nn2 = forward_pass(t_data)
    data_loss = lambda4 * MSE_func(theta_nn2, theta_data)
    
    return ODE_loss + IC_loss + BC_loss + data_loss

# Initialize a list to store the loss values
loss_values_pinn = []

# Start the timer
start_time = time.time()

# Training the neural network
for i in range(training_iter):

    optimizer.zero_grad()   # clear gradients for next train

    # input x and predict based on x
    loss = PINNLoss(theta_pinn, t_test, t_data, theta_data)

    # Append the current loss value to the list
    loss_values_pinn.append(loss.item())

    if i % 1000 == 0:  # print every 100 iterations
        print(f"Iteration {i}: Loss {loss.item()}")

    loss.backward() # compute gradients (backpropagation)
    optimizer.step() # update the ANN weigths

# Stop the timer and calculate the elapsed time
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time} seconds")

Nuevamente, graficamos los resultados

In [ ]:
theta_pred_pinn = theta_pinn(t_test)

print(f'Relative error: {relative_l2_error(theta_pred_pinn, theta_test)}')

plot_comparison(t_test, theta_num, theta_pred_pinn, loss_values_pinn)

## 5. Comparación

In [ ]:
plot_comparison(t_test, theta_num, theta_pred_ann, loss_values_ann)
plot_comparison(t_test, theta_num, theta_pred_pinn, loss_values_pinn)

## **Ejercicios**:

1. Elimina la pérdida de los datos del entrenamiento de la PINN. ¿Aún se puede obtener la solución?
2. Incrementa y reduce el parámetro `std_deviation` en la sección [Preparación de los datos de entrenamiento](#data_prep) para cambiar el `SNR`. También cambia las variables `resample` y `ctime`, y compara los resultados tras entrenar la ANN y la PINN.
3. Ajusta los valores de los parámetros `lambdas` en la función de pérdida para ambas redes y analiza su impacto.
4. Modifica la tasa de aprendizaje (`learning_rate`) del optimizador y el número de iteraciones de entrenamiento, y evalúa el efecto en el desempeño.
5. Cambia el número de capas ocultas (`hidden_layers`), neuronas y funciones de activación de la red neuronal, y observa el impacto en los resultados.

## **Preguntas**:

1. **¿Cómo se puede abordar el sobreajuste en las PINNs si el objetivo es aprender los operadores subyacentes del sistema físico?**  
   <details>
   <summary>Respuesta</summary>
   El sobreajuste en las PINNs (Physics-Informed Neural Networks) es un tema ampliamente discutido. En el aprendizaje automático tradicional, el sobreajuste se asocia con la incapacidad de un modelo para generalizar a datos no vistos previamente, lo que afecta su capacidad para realizar predicciones precisas en nuevas entradas. Sin embargo, en el contexto de las PINNs, buscamos un comportamiento diferente: que la red neuronal funcione como un modelo surrogate de la solución de las ecuaciones diferenciales que describen el sistema físico. En este caso, el sobreajuste no necesariamente es perjudicial, ya que queremos que la red reproduzca fielmente la solución dentro del dominio especificado.

   El desafío relacionado con la generalización en las PINNs surge cuando estas redes se evalúan en geometrías más complejas o diferentes de las que fueron utilizadas durante el entrenamiento. En tales casos, una PINN que esté sobreajustada a una única geometría podría fallar al adaptarse a las nuevas configuraciones, comprometiendo su capacidad para generalizar y capturar las dinámicas físicas subyacentes en contextos más diversos. Por ello, abordar el sobreajuste implica equilibrar la fidelidad al dominio original y la adaptabilidad a nuevas geometrías o condiciones.
   </details>


2. **Qué ventajas ofrece el uso del MSE en la función de pérdida, dado que este enfoque puede subestimar la solución al no incorporar formulaciones integrales o variacionales**  
   <details>
   <summary>Respuesta</summary>
   El uso del MSE como función de pérdida proporciona una forma sencilla y computacionalmente eficiente de medir las diferencias puntuales entre las predicciones del modelo y los valores objetivo. Sin embargo, el MSE se centra únicamente en puntos individuales, lo que puede limitar su capacidad para capturar el comportamiento global de la solución, especialmente en dominios complejos. Al incorporar formulaciones integrales o variacionales, la función de pérdida puede reflejar el comportamiento de la solución en todo el dominio, mejorando potencialmente la precisión y estabilidad. A pesar de estas limitaciones, el MSE sigue siendo popular porque simplifica la implementación y reduce los costos computacionales, lo que lo hace adecuado para muchas aplicaciones prácticas.
   </details>


3. **¿En qué formas son ventajosas las PINNs comparadas con los métodos numéricos tradicionales, considerando el mayor tiempo requerido para el entrenamiento?**  
   <details>
   <summary>Respuesta</summary>
   Las PINNs ofrecen varias ventajas frente a los métodos numéricos tradicionales, a pesar de su tiempo de entrenamiento generalmente más largo. Una ventaja clave es su flexibilidad para manejar dominios complejos, de alta dimensionalidad y geometrías irregulares sin requerir mallas estructuradas. Las PINNs también pueden incorporar fácilmente restricciones o datos adicionales, como mediciones experimentales o condiciones de frontera. A diferencia de muchos métodos numéricos, una vez entrenadas, las PINNs generalizan bien a diferentes condiciones iniciales y de frontera, lo que puede hacerlas más versátiles en escenarios que requieren simulaciones repetidas o ajustes de parámetros. Esta flexibilidad y adaptabilidad las convierten en una herramienta poderosa para ciertas tareas de modelado informadas por física donde los métodos tradicionales pueden estar limitados.
   </details>
